#  import Library

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'tensorflow'

# Load Data

df = pd.read_csv('data/kurs_usd_idr.csv', parse_dates=['date'], index_col='date')
df = df[['rate']]
df.plot(figsize=(12,4), title='Kurs USD/IDR')
plt.grid()
plt.show()

# Preprocessing

In [10]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

def create_dataset(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

window_size = 60
X, y = create_dataset(scaled, window_size)
X = X.reshape(X.shape[0], X.shape[0], 1)

# split train test